# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews


In [4]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

,reviews
0,"✅ Trip Verified | Four very pleasant, on time..."
1,Not Verified | We travel extensively over 25...
2,✅ Trip Verified | Flight delayed an hour due ...
3,✅ Trip Verified | A very full flight made Pre...
4,✅ Trip Verified | The worst airline I’ve ever ...


In [78]:
df.to_csv("BA_reviews.csv")

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [79]:
df = pd.read_csv('BA_reviews.csv')
df

,Unnamed: 0,reviews,verified
0,0,"['fourverypleasant', 'ontimeflightswithfriendl...",NaN
1,1,"['notverified', 'operationalissues', 'flightca...",NaN
2,2,['flightdelayedanhourduetobadweatheraroundjfkh...,NaN
3,3,['averyfullflightmadepremiumeconomyfeelalittle...,NaN
4,4,"['theworstairlinei', 'veeverflown', 'couldnotc...",NaN
...,...,...,...
995,995,['iwasseatedrightnexttothegalleyandlavatoryint...,NaN
996,996,"['flightwasok', 'minordelaydidn', 'tbothme', '...",NaN
997,997,"['gatwicktobarbadosreturn', 'inormallytravelon...",NaN
998,998,"['marseilletolondon', 'iamgenerallyannoyedwith...",NaN


#### Data cleaning

In [80]:
df.columns

Index(['Unnamed: 0', 'reviews', 'verified'], dtype='object')

In [81]:
# removing the unnamed columns
df = df.drop(columns=['Unnamed: 0'])

In [82]:
# look for verified Trip
df['verified'] = df['reviews'].str.contains('Trip Verified')
df['verified']

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: verified, Length: 1000, dtype: bool

In [83]:
df['verified'].value_counts()

verified
False    1000
Name: count, dtype: int64

In [58]:
# removing the Trip verified
df['reviews'] = df['reviews'].str.replace(r'✅ Trip Verified|\s*', '', regex=True)

In [59]:
df

,reviews
0,"|Fourverypleasant,ontimeflightswithfriendly,he..."
1,NotVerified|Wetravelextensivelyover25yearsandt...
2,|FlightdelayedanhourduetobadweatheraroundJFKho...
3,|AveryfullflightmadePremiumEconomyfeelalittlel...
4,|TheworstairlineI’veeverflown.Couldnotcheckinu...
...,...
995,|Iwasseatedrightnexttothegalleyandlavatoryinth...
996,"|FlightwasOK,minordelaydidn'tbothme.Whatwasast..."
997,|GatwicktoBarbadosreturn.InormallytravelonBAwi...
998,|MarseilletoLondon.Iamgenerallyannoyedwiththeq...


In [38]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

In [37]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [60]:
stop_words = set(stopwords.words('english'))

punctuation = set(string.punctuation)
numeric_chars = set(string.digits)

lemmatizer = WordNetLemmatizer()
token = word_tokenize

def preprocessing_text(text):
    # Split text based on punctuation
    segments = re.split(r'[{}]+'.format(re.escape(string.punctuation)), text.lower())
    
    # Tokenize each segment and process
    processed_tokens = []
    for segment in segments:
        if segment.strip():  # Check if segment is not empty
            # Tokenize the segment into words
            tokens = word_tokenize(segment)
            
            # Remove stopwords, punctuation, and lemmatize
            clean_tokens = [
                lemmatizer.lemmatize(word) for word in tokens
                if word.isalpha() and word not in stop_words and word not in punctuation
            ]
            
            # Add processed tokens to the final list
            processed_tokens.extend(clean_tokens)
    
    # Join tokens back into a clean text string
    clean_text = ' '.join(processed_tokens)
    
    return clean_text


In [61]:
# apply the the clean fun to df
df['reviews'] = df['reviews'].apply(preprocessing_text)
df

,reviews
0,fourverypleasant ontimeflightswithfriendly hel...
1,notverified operationalissues flightcancellati...
2,flightdelayedanhourduetobadweatheraroundjfkhow...
3,averyfullflightmadepremiumeconomyfeelalittlele...
4,theworstairlinei veeverflown couldnotcheckinun...
...,...
995,iwasseatedrightnexttothegalleyandlavatoryinthe...
996,flightwasok minordelaydidn tbothme whatwasasto...
997,gatwicktobarbadosreturn inormallytravelonbawit...
998,marseilletolondon iamgenerallyannoyedwiththequ...


In [71]:
pip install pyenchant

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pyenchant from https://files.pythonhosted.org/packages/49/96/2087455de16b08e86fa7ce70b53ddac5fcc040c899d9ebad507a0efec52d/pyenchant-3.2.2-py3-none-win_amd64.whl.metadata
  Using cached pyenchant-3.2.2-py3-none-win_amd64.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.9 MB 325.1 kB/s eta 0:00:37
   ---------------------------------------- 0.0/11.9 MB 279.3 kB/s eta 0:00:43
   ---------------------------------------- 0.1/11.9 MB 476.3 kB/s eta 0:00:25
   ---------------------------------------- 0.1/11.9 MB 514.3 kB/s eta 0:00:23
    --------------------------------------- 0.2/11.9 MB 692.9 kB/s eta 0:00:17
    --------------------------------

In [76]:
import enchant
import re

# Initialize English dictionary from enchant
english_dict = enchant.Dict("en_US")

def separate_combined_words(text):
    # Split text into words
    words = text.split()
    
    # List to store separated words
    separated_words = []
    
    for word in words:
        # Check if the word is not recognized by the dictionary
        if not english_dict.check(word):
            # Use regex to split combined word into potential components
            components = re.findall('[A-Z][^A-Z]*', word)
            # Add components to separated words list
            separated_words.extend(components)
        else:
            separated_words.append(word)
    
    # Join separated words into clean text
    clean_text = ' '.join(separated_words)
    
    return clean_text

# Example usage:
text = "fourverypleasant ontimeflightswithfriendly hel... 1 notverified operationalissues flightcancellati... 2 flightdelayedanhourduetobadweatheraroundjfkhow... 3 averyfullflightmadepremiumeconomyfeelalittlele... 4 theworstairlinei veeverflown couldnotcheckinun... ... 995 iwasseatedrightnexttothegalleyandlavatoryinthe"

cleaned_text = separate_combined_words(text)
print(cleaned_text)


1 2 3 4 ... 995


In [73]:
# apply the the clean fun to df
df['reviews'] = df['reviews'].apply(extract_combined_words)
df

,reviews
0,"[fourverypleasant, ontimeflightswithfriendly, ..."
1,"[notverified, operationalissues, flightcancell..."
2,[flightdelayedanhourduetobadweatheraroundjfkho...
3,[averyfullflightmadepremiumeconomyfeelalittlel...
4,"[theworstairlinei, veeverflown, couldnotchecki..."
...,...
995,[iwasseatedrightnexttothegalleyandlavatoryinth...
996,"[flightwasok, minordelaydidn, tbothme, whatwas..."
997,"[gatwicktobarbadosreturn, inormallytravelonbaw..."
998,"[marseilletolondon, iamgenerallyannoyedwiththe..."


In [88]:
df['reviews'][0]

"['fourverypleasant', 'ontimeflightswithfriendly', 'helpfulstaff', 'theguyonthelhr', 'sfolegwasexceptionalandiregretthatihaveforgottenhisname', 'onlynegativewasthefoodonthesfo', 'lhrleg', 'theadvertisedfiletsteakwasmoreakintostewingsteakandwasinedible', 'sadlythisisnottheonlyoccasionihavefoodonus', 'ukflightstobesubstandard', 'otherwisehappytorecommendba']"